In [1]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

run_name = "noble-field-7"

In [2]:
run = wandb.init()
links = {
    "GOOD_AVERAGE_glamorous-sweep-62": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v283",
    "GOOD_azure-sweep-54": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v279",
    "GOOD_apricot-sweep-17": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v242",
    "GOOD_hearty-sweep-60": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v280",
    "GOOD_worldly-sweep-22": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v245",
    "GOOD_legendary-sweep-5": "mmil_vae_g2d/voice_distribution_and_genre_distribution_imbalance/model_epoch_100:v230",
    "drawn_river_6": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v2",
    "worldly-firebrand-5": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v1",
    "noble-field-7": "mmil_vae_g2d/beta_annealing_study/model_epoch_100:v3",
    "young-violet-12": "mmil_vae_g2d/beta_annealing_study/model_epoch_700:v0",
    "kind-gorge-14": "mmil_vae_g2d/beta_annealing_study/model_epoch_500:v1"
    
}
artifact = run.use_artifact(links[run_name], type='model')
artifact_dir = artifact.download()

In [3]:
import sys
sys.path.insert(0, "../..")

In [4]:
from helpers import load_variational_mgt_model
from model import GrooveTransformerEncoderVAE
import torch

Holoviews not installed, will not be able to generate violin plots


In [5]:
from helpers import load_variational_mgt_model   
import os

In [6]:
model = load_variational_mgt_model(os.path.join(artifact_dir, "100.pth"))

Offset activation is sigmoid, bias is initialized to 0.5


In [7]:
from data import load_gmd_hvo_sequences

train_set = load_gmd_hvo_sequences(
    dataset_setting_json_path = "../../data/dataset_json_settings/4_4_Beats_gmd.json", 
    subset_tag = "validation", 
    force_regenerate=False)

INFO:data.Base.dataLoaders:Loading gmd dataset
INFO:data.Base.dataLoaders:Loading Cached Version from: data/gmd/resources/cached/beat_division_factor_[4]/drum_mapping_label_ROLAND_REDUCED_MAPPING/beat_type_['beat']_time_signature_['4-4']


In [8]:
gt_sample = train_set[0]
groove = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)
gt_sample.metadata

{'Source': 'Groove MIDI Dataset',
 'drummer': 'drummer1',
 'session': 'session1',
 'loop_id': 'drummer1/session1/126:000',
 'master_id': 'drummer1/session1/126',
 'style_primary': 'funk',
 'style_secondary': 'fast',
 'bpm': '125',
 'beat_type': 'beat',
 'time_signature': '4-4',
 'full_midi_filename': 'drummer1/session1/126_funk-fast_125_beat_4-4.mid',
 'full_audio_filename': 'drummer1/session1/126_funk-fast_125_beat_4-4.wav'}

In [9]:
mu, logvar = model.encode_to_mu_logvar(groove)
latent_z = model.reparametrize(mu, logvar)

In [10]:
latent_z

tensor([[-0.0052, -0.6820,  1.2390,  0.8765, -0.9415,  0.9494, -0.4929, -0.6041,
          0.8772, -0.1030, -0.2360,  2.3747, -0.7919,  1.3070, -0.6810,  1.3448,
         -0.1088, -1.6643, -1.3909, -2.1342,  0.3421, -0.0418,  0.3715, -0.3329,
         -1.0179,  0.4151, -1.6757,  0.3201,  0.9378,  0.7509, -0.0368, -0.3612,
         -0.4060, -1.8539, -1.0287,  0.6902, -1.1592, -0.2378,  1.1590,  0.3978,
          1.4986,  0.3111, -0.0743,  1.1318,  0.2518,  0.3274, -1.2252,  0.6719,
         -1.1478,  0.0074, -0.9115, -0.6538, -0.3977,  0.6778, -0.7384, -0.7702,
         -1.3429, -0.4351,  0.4656, -0.1526,  0.6464, -0.2992,  0.0677, -0.3189,
         -0.2750, -0.4605,  1.0278, -0.0960,  0.2370,  0.2947, -0.8604, -0.4444,
         -0.0551, -2.1450, -0.4002,  0.2504, -1.6611,  0.5196,  0.4965,  0.6247,
         -1.1594, -0.3657,  0.2395,  0.1036, -2.1651,  1.0658, -0.9859,  0.8188,
         -2.8615, -0.2799, -0.3118,  0.3485, -0.1742, -2.6051,  1.3875,  0.3121,
         -1.0014,  0.8959, -

In [11]:
voice_thresholds = [0.5] * 9
voice_max_count_allowed = [32] * 9
h, v, o = model.sample(latent_z=latent_z,
                       voice_thresholds=voice_thresholds,
                       voice_max_count_allowed=voice_max_count_allowed,
                       return_concatenated=False,
                       sampling_mode=0)
print(h, v, o)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 1.],
         [0.,

In [12]:
# from sklearn.datasets import load_digits
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler


In [13]:
import numpy as np

latents = []
labels = []
metadatas = []
use_all_styles = True
empty_hvo_seqs = []
for gt_sample in train_set:
    if (use_all_styles or (gt_sample.metadata["style_primary"] in ["rock", "funk", "afrobeat"])):
        empty_hvo_seqs.append(gt_sample.copy_empty())
        metadatas.append(gt_sample.metadata)
        labels.append(gt_sample.metadata["style_primary"])
        groove = torch.zeros((1, 32, 3))
        flattened_ = torch.tensor([gt_sample.flatten_voices(reduce_dim=True)], dtype=torch.float32)[:,:32, :]
        t_steps = flattened_.shape[1]
        groove[:, :t_steps, :] = flattened_
        mu, logvar = model.encode_to_mu_logvar(groove)
        latent_z = model.reparametrize(mu, logvar)
        latents.append(latent_z.detach().cpu().numpy())

latents = np.array(latents).squeeze(1)
features = np.expand_dims(latents, -1) # we use each dimension of latent_z as a feature
feature_labels = [f"z_{dim}" for dim in range(features.shape[1])]

In [14]:
import pandas as pd

data = {"style_primary": [style for style in labels]}
data.update({f"z_{dim_i}": latents[:, dim_i] for dim_i in range(features.shape[1])})
df = pd.DataFrame(data)
df.head()

,style_primary,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8,...,z_118,z_119,z_120,z_121,z_122,z_123,z_124,z_125,z_126,z_127
0,funk,1.405968,-1.418204,2.241938,-1.432247,-0.244208,1.670453,0.099018,-0.378574,0.409897,...,-0.592854,0.718838,-0.723409,1.814216,0.642250,-0.178092,-1.645441,0.545067,1.054374,0.235548
1,funk,-0.514229,-1.129054,1.198230,0.151740,-1.007104,-0.173251,0.217129,0.882369,-0.343454,...,0.119521,0.122811,1.465395,1.117042,1.133844,-0.352000,-1.300792,-0.515341,-0.256571,0.459162
2,funk,-0.896100,-2.112238,1.284186,-0.292598,-1.819460,-0.696763,-1.099944,-0.880708,0.834336,...,-0.220998,-1.204104,-0.742718,1.658747,-1.456713,-0.865167,-2.156501,-0.570670,-1.736798,-0.758118
3,funk,-0.372628,-1.616554,1.406586,-1.358405,-1.634862,-1.031933,-1.341431,-0.207904,0.598058,...,-0.015780,0.210687,0.800610,-0.624901,1.850504,-0.656793,-0.617875,-0.585567,-1.337538,0.327894
4,funk,0.030881,-2.150223,0.975269,0.989192,-1.282974,-0.910142,-0.546898,0.221775,0.348787,...,0.904937,-1.712473,-0.130200,1.144169,0.667342,-0.243410,-0.969281,0.094148,-1.536754,-0.339237


In [15]:
from bokeh.io import output_notebook
from bokeh.plotting import show
import IPython.display as ipd
import sys
# sys.path.insert(0, '/usr/local/bin/fluidsynth')
output_notebook()

Loading BokehJS ...

In [25]:
voice_thresholds=[0.3]*9
voice_thresholds[1] = 0.5
voice_thresholds[3] = 0.1

# voice_thresholds[-2] = 0.01
# voice_thresholds[-3] = 0.01

# random_z = [np.random.uniform(df[f"z_{i}"].min(), df[f"z_{i}"].max()) for i in range(len(latents[0, :]))]
# random_z = [np.random.uniform(df[f"z_{i}"].min(), df[f"z_{i}"].max()) for i in range(len(latents[0, :]))]
random_z = [np.random.normal(loc=df[f"z_{i}"].mean(), scale=df[f"z_{i}"].std()) for i in range(len(latents[0, :]))]


hvo = model.sample(latent_z=torch.tensor(random_z, dtype=torch.float32), 
                   voice_thresholds=voice_thresholds,
                   voice_max_count_allowed=[32]*9,
                   return_concatenated=True,
                   sampling_mode=0)
hvo_seq_ = empty_hvo_seqs[0]
hvo_seq_.hvo = hvo.detach().cpu().numpy()[0]

if (hvo_seq_.get_number_of_active_voices()>0):
    # draw and synthesize
    show(hvo_seq_.piano_roll())
    audio = hvo_seq_.synthesize(sf_path="../../hvo_sequence/soundfonts/TamaRockSTAR.sf2")

else:
    print("Empty Score")
    
ipd.Audio(audio, rate=44100, autoplay=True) # load a NumPy array

fluidsynth: error: Unknown integer parameter 'synth.sample-rate'
